## Elastic search in python 1
for more info
- code: https://github.com/mageirakos/elasticsearch-movies
- data source: https://grouplens.org/datasets/movielens/

In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [2]:
import os
import pandas as pd


In [3]:
path='/home/mn/Courses/ElasticSearch/ml-latest-small'
file='movies.csv'
file_name=os.path.join(path, file)

In [4]:
df_movies=pd.read_csv(file_name)

In [5]:
df_movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [7]:
from elasticsearch import helpers, Elasticsearch
import csv

with open(file_name, encoding='utf-8') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='movies')

/tmp/ipykernel_4474/3370894933.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es, reader, index='movies')


In [8]:
def queryInput(user_input):
    res = es.search(index="movies", body={"query": {"match": {"title":"{}".format(user_input)}}}, size = 1000)
    hits = res['hits']['total']['value']
    print("Got {} Hits:".format(hits))

    # I set the maximum number of results to 1000. The default was 10. So we need to take that into account
    try :
        for i in range(hits):
            print(i+1,') ',res['hits']['hits'][i]['_source']['title'])
    except:
        for i in range(1000):
            print(i+1,') ',res['hits']['hits'][i]['_source']['title'])
            
    return

In [9]:
def queryInput2(user_input_title):
    ''' returns the movie_Id, title and genres '''
    # ps = per_search since BM25 _score changes depending on user_input
    result_params_list_df_ps = []

    res = es.search(index="movies", body={"query": {"match": {"title":"{}".format(user_input_title)}}}, size = 1000)
    #res2 = es.search(index="movies", body={"query": {"match": {"content":"{}".format(user_input)}}}, size = 1000)
    hits = res['hits']['total']['value']
    #hits= res2['hits']['total']['value']
    print("Got {} Hits:".format(hits))

    # I set the maximum number of results to 1000. The default was 10. So we need to take that into account
    try :
        for i in range(hits):          
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
            
    except:
        for i in range(1000):
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
    lst = result_params_list_df_ps
    return  print(lst)

In [10]:
print('type "//exit" if you want to exit the search')
user_input= input("Which movie do you want? (by title): \n")

while(user_input != '//exit'):
    queryInput2(user_input)
    print('type "//exit" if you want to exit the search')
    user_input = input("Which movie do you want? (by title): \n")
    

type "//exit" if you want to exit the search


Which movie do you want? (by title): 
 interstellar


/tmp/ipykernel_4474/1002140731.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="movies", body={"query": {"match": {"title":"{}".format(user_input_title)}}}, size = 1000)
/tmp/ipykernel_4474/1002140731.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="movies", body={"query": {"match": {"title":"{}".format(user_input_title)}}}, size = 1000)


Got 4 Hits:
[   movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX]
type "//exit" if you want to exit the search


Which movie do you want? (by title): 
 //exit


This time with match-Phrase:

In [18]:
def queryInput3(user_input_title):
    ''' returns the movie_Id, title and genres '''
    # ps = per_search since BM25 _score changes depending on user_input
    result_params_list_df_ps = []

    res = es.search(index="movies", body={"query": {"match_phrase": {"title":"{}".format(user_input_title)}}}, size = 1000)
    #res2 = es.search(index="movies", body={"query": {"match": {"content":"{}".format(user_input)}}}, size = 1000)
    hits = res['hits']['total']['value']
    #hits= res2['hits']['total']['value']
    print("Got {} Hits:".format(hits))

    # I set the maximum number of results to 1000. The default was 10. So we need to take that into account
    try :
        for i in range(hits):          
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
            
    except:
        for i in range(1000):
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
    lst = result_params_list_df_ps
    return  print(lst)

In [20]:
print('type "//exit" if you want to exit the search')
user_input= input("Which movie do you want? (by title): \n")

while(user_input != '//exit'):
    queryInput3(user_input)
    print('type "//exit" if you want to exit the search')
    user_input = input("Which movie do you want? (by title): \n")
    

type "//exit" if you want to exit the search


Which movie do you want? (by title): 
 interstellar


/tmp/ipykernel_4474/87324818.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="movies", body={"query": {"match_phrase": {"title":"{}".format(user_input_title)}}}, size = 1000)
/tmp/ipykernel_4474/87324818.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="movies", body={"query": {"match_phrase": {"title":"{}".format(user_input_title)}}}, size = 1000)


Got 4 Hits:
[   movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX,    movieId                title       genres
0   109487  Interstellar (2014)  Sci-Fi|IMAX]
type "//exit" if you want to exit the search


Which movie do you want? (by title): 
 //exit


In [21]:
def queryInput3(user_input_title, user_input_genre):
    ''' returns the movie_Id, title and genres '''
    # ps = per_search since BM25 _score changes depending on user_input
    result_params_list_df_ps = []

    res = es.search(index="movies", body={
        "mappings": {"properties": {
            "genres":"{}".format(user_input_genre),
            "title":"{}".format(user_input_title)}}}, size = 1000)
    
    hits = res['hits']['total']['value']
    #hits= res2['hits']['total']['value']
    print("Got {} Hits:".format(hits))

    # I set the maximum number of results to 1000. The default was 10. So we need to take that into account
    try :
        for i in range(hits):          
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
            
    except:
        for i in range(1000):
            temp_df = pd.DataFrame([ [ int(res['hits']['hits'][i]['_source']['movieId']), res['hits']['hits'][i]['_source']['title'], res['hits']['hits'][i]['_source']['genres'] ] ], columns=['movieId', 'title', 'genres'])
            result_params_list_df_ps.append(temp_df)
    lst = result_params_list_df_ps
    return  print(lst)

In [23]:
print('type "//exit" if you want to exit the search')
user_input_title= input("Which movie do you want? (by title): \n")
user_input_genre= input("Which movie do you want? (by genre): \n")
while(user_input != '//exit'):
    queryInput3(user_input_title, user_input_genre)
    print('type "//exit" if you want to exit the search')
    user_input_title = input("Which movie do you want? (by title): \n")
    user_input_genre= input("Which movie do you want? (by genre): \n")

type "//exit" if you want to exit the search


Which movie do you want? (by title): 
 interstellar
Which movie do you want? (by genre): 
 Sci-Fi


In [24]:
import requests
response=requests.get("http://localhost:9200/_cat/indices")
print(response.text)

yellow open tutorial-1           -Z9W6iLUTHmegu0QZyGW6g 1 1      0  0   226b   226b
yellow open tutorieal-1          ExCC-P1yRSyTt30eFhpvoQ 1 1      0  0   226b   226b
yellow open hr                   lS8BE3xVT3Ok_5Fbw6pyIw 1 1      0  0   226b   226b
yellow open employee             VGLlHPRSShSJVFVH0yinkQ 1 1      0  0   226b   226b
yellow open sales                69LDlmrgQYObdQKb5hwIKg 1 1      0  0   226b   226b
yellow open index_1              M6xBjdnkT3iVaIMS39orEg 1 1      0  0   226b   226b
yellow open movies               NpvpjcETQbqmnhZIpQodww 1 1  38968  0  5.4mb  5.4mb
green  open .geoip_databases     4D6A68zIS6WcupHXn1D40Q 1 0     40 37 37.8mb 37.8mb
yellow open index_2              tJU1JLx0TpKZ-MCHL8eQRQ 1 1      0  0   226b   226b
yellow open test-index           lcKzDyY9RcuXRrbrVPlOog 1 1      1  0  3.9kb  3.9kb
yellow open index_3              m_GPK56ARleLwY9jaeQn9Q 1 1      0  0   226b   226b
yellow open index_4              y7NmrmRHQiixhGWZ4cloww 1 1      0  0   226b